The aim of this notebook is to benchmark different methods such as KDTree, brute force, Gridsearch (CellGrid and NeighbourSearch) for a real case. The files for the data are obtained from [here](https://github.com/richardjgowers/atomgroup_olympics). The dataset is primarily molecules of water in a box, so ideally a set of three particles should be bonded to each other.

The previous comparisons for guessing the bonds (see [here](https://github.com/ayushsuhane/Benchmarks_Distance/blob/master/Notebooks/GuessBonds/GuessBonds.ipynb)) are used with little modification for this benchmark.

First, lets begin by defining functions to guess the bonds.

In [1]:
import numpy as np
import MDAnalysis as mda
import MDAnalysis.topology.guessers as mda_bf
from MDAnalysis.topology import tables
from MDAnalysis.lib.pkdtree import PeriodicKDTree
from MDAnalysis.lib.distances import distance_array
import cellgrid

# Cellgrid

In [2]:
def cg_guessbonds(points, atoms = None, box = None):
    """
    Guessing Bonds with cell-List data structures 
    as implemented in Cellgrid Structure
    """ 
    if atoms is None:
        ### Use default values
        vdw = 2.0
        atoms = np.ones(len(points), dtype = np.float32)*vdw
    else:
        ##First check if every atom radius correspond to one particle
        if len(points) != len(atoms):
            print("Atoms and Coordinates should have equal length")
            return
        
    fudge_factor = 0.72
    lower_bound = 0.1
    
    bonds = []
    vdwradii = tables.vdwradii.copy()
    max_vdw = max([vdwradii[t] for t in atoms])
    
    indx,dist = cellgrid.capped_self_distance_array(points, 2.0*max_vdw*fudge_factor , box=box[:3], particles_per_cell = False)
    mask = np.where((dist <= (2.0*max_vdw)*fudge_factor) & (dist > lower_bound))[0]
    for num, (i,j) in enumerate(indx[mask]):
        if dist[mask][num] < ((vdwradii[atoms[i]] + vdwradii[atoms[j]]) * fudge_factor):
            bonds.append((i,j))  
    return tuple(bonds)

# KDTree

In [3]:
def kd_guessbonds(points, atoms = None, box = None):
    """
    Already implemented tree structure in MDAnalysis
    """
    if atoms is None:
        ### Use default values
        vdw = 2.0
        atoms = np.ones(len(points), dtype = np.float32)*vdw
    else:
        ##First check if every atom radius correspond to one particle
        if len(points) != len(atoms):
            print("Atoms and Coordinates should have equal length")
            return
        
    fudge_factor = 0.72
    lower_bound = 0.1

    bonds = []
    
    vdwradii = tables.vdwradii.copy()
    max_vdw = max([vdwradii[t] for t in atoms])
    
    kdtree = PeriodicKDTree(box, bucket_size=10)
    kdtree.set_coords(points)
    
    for idx,centers in enumerate(points):
        vdw_i = vdwradii[atoms[idx]]
        max_d = (vdw_i + max_vdw)*fudge_factor
        kdtree.search(centers,max_d)
        indices = kdtree.get_indices()
        dist = distance_array(centers.reshape((1,3)), points[indices],box=box)[0]
        index = np.where((dist > lower_bound) & (dist <= max_d))[0]
        for j in index:
            if (indices[j] > idx) and (dist[j] < (vdwradii[atoms[indices[j]]] + vdw_i)*fudge_factor):
                bonds.append((idx, indices[j])) 
    return tuple(bonds)

# FATSLiM

In [4]:
# Neighbour search FATSLiM
from core_ns import FastNS
def ns_guessbonds(points, atoms = None, box = None):
    if atoms is None:
        ### Use default values
        vdw = 2.0
        atoms = np.ones(len(points), dtype = np.float32)*vdw
    else:
        ##First check if every atom radius correspond to one particle
        if len(points) != len(atoms):
            print("Atoms and Coordinates should have equal length")
            return
        
    fudge_factor = 0.72
    lower_bound = 0.1

    bonds = []
    
    vdwradii = tables.vdwradii.copy()
    max_vdw = max([vdwradii[t] for t in atoms])
    
    
    triclinic_box = np.array([[box[0], 0, 0],[0, box[1], 0],[0,0,box[2]]], dtype=np.float32)
    searcher = FastNS(triclinic_box)
    searcher.set_cutoff(2*max_vdw)
    searcher.set_coords(points)
    searcher.prepare()
    for idx, centers in enumerate(points):
        vdw_i = vdwradii[atoms[idx]]
        max_d = (vdw_i + max_vdw)*fudge_factor
        
        results, sqdist, index = searcher.search(np.array([centers,]))
        if len(index[0]) > 0:
            for i, val in enumerate(index[0]):
                d = (vdwradii[atoms[val]] + vdw_i)*fudge_factor
                if (sqdist[0][i] < d*d) and (val > idx):
                    bonds.append((idx, val))
    return bonds

# Octree

In [5]:
import pcl
def oct_guessbonds(points, atoms = None, box = None):
    
    fudge_factor = 0.72
    lower_bound = 0.1

    bonds = []
    
    vdwradii = tables.vdwradii.copy()
    max_vdw = max([vdwradii[t] for t in atoms])
    
    cloud = pcl.PointCloud(points)
    resolution = 0.1*box[0]
    octree = cloud.make_octreeSearch(resolution)
    octree.add_points_from_input_cloud()
    bonds = []
    
    for idx, centers in enumerate(points):
        vdw_i = vdwradii[atoms[idx]]
        max_d = (vdw_i + max_vdw)*fudge_factor
        
        [ind, sqdist] = octree.radius_search (centers, 2*max_vdw)
        
        if len(ind) != 0 :
            for i, val in enumerate(ind):
                d = (vdwradii[atoms[val]] + vdw_i)*fudge_factor
                if (sqdist[i] < d*d) and (val > idx):
                    bonds.append((idx, val))
           
    return bonds

The first step is to read the files small.gro and big.gro 

In [6]:
u1,u2 = mda.Universe('small.gro'), mda.Universe('big.gro')

First check the output of MDAnalysis, which utilizes brute force algorithm.

In [7]:
len(mda_bf.guess_bonds(u1.atoms, u1.atoms.positions, u1.dimensions))

8284

Lets evaluate the pair contacts due to all the functions defined above. 

In [46]:
len(ns_guessbonds(u1.atoms.positions, u1.atoms.types, u1.dimensions[:3]))

8284

In [297]:
len(oct_guessbonds(u1.atoms.positions, u1.atoms.types, u1.dimensions[:3]))

8284

In [298]:
len(kd_guessbonds(u1.atoms.positions, u1.atoms.types, u1.dimensions))

8284

In [301]:
len(cg_guessbonds(u1.atoms.positions, u1.atoms.types, u1.dimensions[:3]))

8180

CellGrid needs to be checked. Even after trying with the option `particles_per_cell = False` returns a wrong result here. Till the time, lets check the performance of other methods.

In [325]:
import itertools
ns = ns_guessbonds(u2.atoms.positions, u2.atoms.types, u2.dimensions[:3])
len(ns)

1043128

In [326]:
octree = oct_guessbonds(u2.atoms.positions, u2.atoms.types, u2.dimensions[:3])
len(octree)

1043128

In [327]:
pkdt = kd_guessbonds(u2.atoms.positions, u2.atoms.types, u2.dimensions)
len(pkdt)

1043128

In [ ]:
bf = mda_bf.guess_bonds(u2.atoms, u2.atoms.positions, u2.dimensions)
len(bf)

In [8]:
from tqdm import tqdm_notebook

In [8]:
# Timings
from collections import defaultdict
from tqdm import tqdm_notebook

result = defaultdict(list)

for u in [u1]:
    #NS
    res = %timeit -q -o -n 5 -r 5 ns_guessbonds(u.atoms.positions, u.atoms.types, u.dimensions[:3])
    result['ns'].append(res.average)
    
    #PKDT
    res = %timeit -q -o -n 5 -r 5 kd_guessbonds(u.atoms.positions, u.atoms.types, u.dimensions)
    result['pkdt'].append(res.average)
    
    #Octree
    res = %timeit -q -o -n 5 -r 5 oct_guessbonds(u.atoms.positions, u.atoms.types, u.dimensions[:3])
    result['oct'].append(res.average)
    
    #BF
    res = %timeit -q -o -n 5 -r 5 mda_bf.guess_bonds(u.atoms, u.atoms.positions, u.dimensions)
    result['bf'].append(res.average)
    

In [29]:
print("{:10} {:10} {:7} {:7} {:7} {:7}".format('Filename', 'NParticles', 'PKDTree', 'Octree', 'FATSLiM', 'BruteForce'))
print("{:10} {:10} {:7.5} {:7.5} {:7.5} {:7.5} ".format('small.gro', len(u1.atoms), result['pkdt'][0], result['oct'][0], result['ns'][0], result['bf'][0]))


Filename   NParticles PKDTree Octree  FATSLiM BruteForce
small.gro       12426  2.0887 0.38741 0.40072  3.5214 


In [32]:
result = defaultdict(list)

In [33]:
res = %timeit -q -o -n 1 -r 1 ns_guessbonds(u2.atoms.positions, u2.atoms.types, u2.dimensions[:3])
result['ns'].append(res.average)

In [36]:
res = %timeit -q -o -n 1 -r 1 kd_guessbonds(u2.atoms.positions, u2.atoms.types, u2.dimensions)
result['pkdt'].append(res.average)

In [38]:
#Octree
res = %timeit -q -o -n 1 -r 1 oct_guessbonds(u2.atoms.positions, u2.atoms.types, u2.dimensions[:3])
result['oct'].append(res.average)

In [ ]:
#Brute Force
res = %timeit -q -o -n 1 -r 1 mda_bf.guess_bonds(u2.atoms, u2.atoms.positions, u2.dimensions)
result['bf'].append(res.average)

In [39]:
result['ns'], result['pkdt'], result['oct']

([54.3090979999979], [236.48744599999918], [151.2018640000024])

In [ ]:
print("{:10} {:10} {:7} {:7} {:7} {:7}".format('Filename', 'NParticles', 'PKDTree', 'Octree', 'FATSLiM', 'BruteForce'))
print("{:10} {:10} {:7.5} {:7.5} {:7.5} {:7.5} ".format('small.gro', len(u2.atoms), result['pkdt'][0], result['oct'][0], result['ns'][0], result['bf'][0]))